In [25]:
#from imblearn.pipeline import Pipeline
#from imblearn.over_sampling import RandomOverSampler
#from imblearn.under_sampling import RandomUnderSampler
import pandas as pd
import numpy as np
import seaborn as sns
from sklearn.preprocessing import LabelEncoder
from sklearn.ensemble import RandomForestClassifier
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression

# Load the data
data = pd.read_csv('hotel_booking.csv')
pre_processed_data=data

In [ ]:
#1. Data Exploration and Preparation

# Get a Summary of Data
describe = data.describe()
print("\n"+ str(describe))

# Get information about the data
hotel_data_info = data.info()
# Size
size = data.size
print('O tamanho do data set é: ' + str(size))

In [ ]:
# View the first rows of the data set and then the last rows
first = data.head()
last = data.tail()
print("The first rows are:\n" + str(first) + "\n The last rows are: \n" + str(last))

In [ ]:
# Check which columns are Numeric and which are Categorical
dtypes = data.dtypes
print("Os data types são: \n" + str(dtypes) )

In [ ]:
# Number of rows
linhas = len(data)
print("Number of rows: " + str(linhas))

# Number of collumns
colunas = len(data.columns)
print("Number of columns: " + str(colunas))

# Collumns Names
col_names = data.columns
print(col_names)

In [ ]:
# Display the number of missing values in each column sorted in ascending order
print(f"Number of nulls per column: {data.isnull().sum().sort_values(ascending=False)}")

print(f"Number of duplicate rows: {data.duplicated().sum()}")


In [ ]:
# Assign a placeholder for missing values
data.fillna(value={'children': -1, 'country': 'Missing', 'agent': 'Missing', 'company': 'No Company'}, inplace=True)
data.info()
print(f"Number of nulls per column: {data.isnull().sum().sort_values(ascending=False)}")


In [ ]:
# Verify if have duplicate rows
print(size)
df_duplicates = data.drop_duplicates(inplace = True)
print(data.size) # Doesn't have duplicate rows

# Check for unique values
unique_values = data.nunique()
print(unique_values)

In [ ]:
# Separate columns into categorical and numerical
categorical_columns = data.select_dtypes(include=['object']).columns.tolist()
numerical_columns = data.select_dtypes(include=['int64', 'float64']).columns.tolist()

# Display the separated columns
print("Categorical Columns:")
print(categorical_columns)
print("\nNumerical Columns:")
print(numerical_columns)

# Check the data types of each column
print(data.dtypes)
data.info()


In [ ]:
# Visualize top 10 countries with reservation canceled
cancelled_data= data[data['is_canceled']==1]
top_10_country = cancelled_data['country'].value_counts()[:15]


plt.figure(figsize=(8, 8))  # Set background color to a light brown
plt.title('Top 10 countries with reservation canceled', color="black")
plt.pie(top_10_country, autopct='%.2f', labels=top_10_country.index,)
plt.show()

In [ ]:
# Get categorical columns

excluded_columns = ['name', 'email', 'phone-number', 'credit_card','reservation_status_date']
categorical_columns = [col for col in data.select_dtypes(include=['object']).columns if col not in excluded_columns]

print("Categorical columns: " + str(categorical_columns))


for column in categorical_columns:
    sns.countplot(data=data, x=column)
    plt.xticks(rotation=45)
    plt.show()


In [ ]:

# Initialize the LabelEncoder
label_encoder = LabelEncoder()

# Convert non-numeric columns to values
for column in categorical_columns:
    # Convert the column to string type to avoid type conflicts
    data[column] = data[column].astype(str)
    # Apply LabelEncoder
    data[column] = label_encoder.fit_transform(data[column])

print(data.info())

In [ ]:
print(data['is_canceled'].info())

In [ ]:
# Let's check the correlation between variables 
numerical_columns = data.select_dtypes(include=['int64', 'float64']).columns.tolist()
correlation_num = data[numerical_columns].corr()
print(correlation_num)

# Create a heatmap
plt.figure(figsize=(12, 10))
sns.heatmap(correlation_num, annot=True, cmap='coolwarm', fmt=".2f", linewidths=.5)
plt.title('Correlation Matrix')
plt.show()

In [ ]:
def univariate_analysis(data, column_name):
    describe= data[column_name].describe()
    variance_value = data[column_name].var()
    iqr_value = data[column_name].quantile(0.75) - data[column_name].quantile(0.25)
    skewness_value = data[column_name].skew()
    kurtosis_value = data[column_name].kurtosis()

    frequency_distribution = data[column_name].value_counts().head(10)

     # Visualization
    plt.figure(figsize=(15, 5))

    # Histogram
    plt.subplot(1, 3, 1)
    sns.histplot(data[column_name], kde=True, bins=30)
    plt.title(f'Histogram of {column_name}')

    # Box Plot
    plt.subplot(1, 3, 2)
    sns.boxplot(x=data[column_name])
    plt.title(f'Box Plot of {column_name}')

   
    # Bar Chart (for categorical features)
    if data[column_name].dtype == 'int' or data[column_name].dtype == 'object':
        plt.subplot(1, 3, 3)
        sns.countplot(x=data[column_name], order=data[column_name].value_counts().index[:10])
        plt.title(f'Bar Chart of {column_name} (Top 10)')


    plt.tight_layout()
    plt.show()

variables_to_analyse=data.columns
for varible in variables_to_analyse:
    univariate_analysis(data, varible)


In [ ]:
# Passo 2: Univariate Analysis (Distribution of individual features)
def bivariate_analysis(data, column_name):
    


    Objective_col = data.columns[1]

    # Visualization
    plt.figure(figsize=(15, 5))

    sns.barplot(data=data, x=Objective_col, y=column_name)
   # plt.savefig(f'bivariate_analysis/bar_plot_{column_name}.png')

    sns.displot(data=data, x= column_name, hue=Objective_col, kind="kde")
    #plt.savefig(f'bivariate_analysis/dis_plot_kde_{column_name}.png')

    fig, axes = plt.subplots(1, 2, figsize=(15, 5), sharey=True)

    # Get unique values in the last_col (your categorical variable)
    unique_types = data[Objective_col].unique()

   
   # Loop over each unique type and perform linear regression
    for i, t in enumerate(unique_types):
        # Filter data for the current type
        subset = data[data[Objective_col] == t]
        
        X = subset[column_name].values.reshape(-1, 1)
        y = subset[column_name].values
        model = LinearRegression()
        model.fit(X, y)
        
        m = model.coef_[0]
        b = model.intercept_
        
        axes[i].scatter(X, y, color='blue', alpha=0.5)
        
        axes[i].plot(X, model.predict(X), color='red', label=f'y = {m:.2f}x + {b:.2f}')
        
        # Set the title to show the current type and regression equation
        axes[i].set_title(f'{t}')
        axes[i].set_xlabel(column_name)

    # Set the common ylabel for the whole figure
    axes[0].set_ylabel('Values')

    # Display the plot
    plt.suptitle(f'Scatter Plots and Regression Lines for Each Type in {Objective_col}')
 

    plt.tight_layout()
    #plt.savefig(f'bivariate_analysis/scatter_plot_{column_name}.png')
    plt.show()



variables_to_analyse=numerical_columns

# Realizar a análise bivariada para cada variável
for variable in variables_to_analyse:
    bivariate_analysis(data, variable)




In [ ]:

categorical_columns = data.select_dtypes(include=['object', 'category']).columns
# Get the number of unique values for categorical columns
unique_values_per_categorical_column = data[categorical_columns].nunique()

# Print the result
print(unique_values_per_categorical_column)

# Function to display the counts of is_canceled for each unique value in the column
def display_cancellation_counts(data, column, top_n=10):
    unique_vals = data[column].nunique()
    
    # Group by the column and the 'is_canceled' column, and count occurrences
    grouped = data.groupby([column, 'is_canceled']).size().unstack(fill_value=0)
    
    if unique_vals <= top_n:
        # If the number of unique values is small, show all counts
        print(f"\nColumn: {column}")
        print(grouped)
    else:
        # If there are many unique values, show the top N values based on their total counts
        top_values = data[column].value_counts().head(top_n).index
        print(f"\nColumn: {column} (Top {top_n} out of {unique_vals} unique values)")
        print(grouped.loc[top_values])

# Apply the function for each column except 'is_canceled'
for column in categorical_columns:
     display_cancellation_counts(data, column)

In [ ]:

def bivariate_categorical(data, varaible):
    plt.figure(figsize=(15, 15))

    sns.countplot(data=data, x=variable, hue=data['is_canceled'])
    # Add labels and title
    plt.xlabel(f'{variable}')
    plt.ylabel('Count')
    plt.title('Count of is_canceled Type by 'f'{variable}')
    plt.legend(title='Is_canceled')


    plt.show()

for variable in categorical_columns:
    bivariate_categorical(data, variable)

   


In [ ]:
print(data.info())
